In [1]:
import pickle
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import combinations_with_replacement
from genova.utils.BasicClass import Ion, Residual_seq

In [ ]:
experiment_name = 'Cerebellum'
psm_head = pd.read_csv('/data/z37mao/genova_new/{}_PSMs.csv'.format(experiment_name),index_col = 'File ID')
psm_head = psm_head[np.logical_and(psm_head['Percolator PEP']<0.05,psm_head['XCorr']>1)]
psm_head = psm_head[psm_head['Annotated Sequence'].str.len()<=32]

In [ ]:
buffer=''
for i, (index, seq) in enumerate(psm_head['Annotated Sequence'].iteritems()):
    buffer+='>{}\n'.format(index)
    buffer+='{}\n'.format(seq.upper())

In [ ]:
def cleavage_find(product_moverz,candidate_moverz,moverz_threshold=0.02):
    return np.array([0 if low_bound==high_bound else 1 for low_bound, high_bound in zip(product_moverz.searchsorted(candidate_moverz-moverz_threshold),product_moverz.searchsorted(candidate_moverz+moverz_threshold))])

In [ ]:
a=[]
b=[]
miss_cleavage_num = []
with open('/data/z37mao/genova_new/{}.mgfs'.format(experiment_name),'rb') as f:
    for i, (seq, charge, pointer, data_len, pep, xcorr) in psm_head[['Annotated Sequence','Charge','MGFS_Datablock_Pointer','MGFS_Datablock_Length','Percolator PEP','XCorr']].iterrows():
        f.seek(pointer)
        seq = seq.replace('L','I')
        product_moverz = pickle.loads(f.read(data_len))['product_ions_moverz']
        seq_composition = Residual_seq.seqs2composition_list(seq)
        seqaamass = np.array([aa_composition.mass for aa_composition in seq_composition])
        a_ion = Ion.sequencemz2ion(seqaamass[:-1].cumsum(),'1a')
        b_ion = Ion.sequencemz2ion(seqaamass[:-1].cumsum(),'1b')
        y_ion = np.flip(Ion.sequencemz2ion(np.flip(seqaamass)[:-1].cumsum(),'1y'))
        y2_ion = np.flip(Ion.sequencemz2ion(np.flip(seqaamass)[:-1].cumsum(),'2y'))
        a_ion = a_ion[a_ion<250]
        y2_ion = y2_ion[y2_ion>400]
        seq_len = len(seq)
        #if seq_len>32: continue
        a_cleavage = np.pad(cleavage_find(product_moverz,a_ion),[0,seq_len-a_ion.size-1])
        b_cleavage = cleavage_find(product_moverz,b_ion)
        y_cleavage = cleavage_find(product_moverz,y_ion)
        if charge>2:
            y2_cleavage = np.pad(cleavage_find(product_moverz,y2_ion),[seq_len-y2_ion.size-1,0])
            cleavage_result = (a_cleavage+b_cleavage+y_cleavage+y2_cleavage)>0
        else:
            cleavage_result = (a_cleavage+b_cleavage+y_cleavage)>0
        noncleavate=np.argwhere(cleavage_result).reshape(-1)[1:]-np.argwhere(cleavage_result).reshape(-1)[:-1]
        a.append(len(seq))
        b.append(np.max(noncleavate))
        miss_cleavage_num.append(np.max(noncleavate)-1)
        #b.append((noncleavate>2).sum()/noncleavate.size)
        #break
        #miss_cleavage_num += list(np.argwhere(cleavage_result).reshape(-1)[1:]-np.argwhere(cleavage_result).reshape(-1)[:-1])
#miss_cleavage_num = np.array(miss_cleavage_num)-1·

In [3]:
psm_head = []
for psm_file_name in glob('/data/z37mao/genova_new/*_PSMs.csv'):
    #if psm_file_name.split('/')[-1][:-9]=='Plasma' or psm_file_name.split('/')[-1][:-9]=='PXD008844':
    #    continue
    psm_head_temp = pd.read_csv(psm_file_name)
    psm_head_temp['File ID'] = psm_file_name.split('/')[-1][:-9]+':'+psm_head_temp['File ID']
    psm_head.append(psm_head_temp)
psm_head = pd.concat(psm_head)
#psm_head = psm_head[np.logical_and(psm_head['Percolator PEP']<0.05,psm_head['XCorr']>1)]
#psm_head = psm_head[psm_head['Annotated Sequence'].str.len()<=32]
#candidate_index = psm_head['File ID']

In [8]:
psm_head = psm_head.drop_duplicates('File ID')

In [16]:
psm_index = []
for psm_file_name in glob('/home/z37mao/temp/*csv'):
    psm_head_temp = pd.read_csv(psm_file_name)
    #psm_head_temp['File ID'] = psm_file_name.split('/')[-1][:-9]+':'+psm_head_temp['File ID']
    psm_index.append(psm_head_temp)
psm_index = pd.concat(psm_index)
psm_index[['Experiment Name','Raw File ID','Spectrum ID']] = psm_index['Spec Index'].str.split(':',expand=True)
psm_index = psm_index.drop_duplicates('Spec Index')
psm_index = psm_index.set_index('Spec Index')

In [17]:
psm_index

,Node Number,Relation Num,Edge Num,MSGP File Name,MSGP Datablock Pointer,MSGP Datablock Length,Experiment Name,Raw File ID,Spectrum ID
Spec Index,,,,,,,,,
Cerebellum:F13.23:13749,111,16785,80167,15_0.msgp,0,410663,Cerebellum,F13.23,13749
Cerebellum:F13.2:32021,176,65745,197302,15_0.msgp,410663,1071094,Cerebellum,F13.2,32021
Cerebellum:F13.16:14029,77,8005,33280,15_0.msgp,1481757,169110,Cerebellum,F13.16,14029
Cerebellum:F8.7:18799,74,8292,33361,15_0.msgp,1650867,175219,Cerebellum,F8.7,18799
Cerebellum:F8.3:46428,108,21796,109977,15_0.msgp,1826086,536535,Cerebellum,F8.3,46428
...,...,...,...,...,...,...,...,...,...
Cerebellum:F8.7:76408,245,209437,473066,20_13.msgp,2637910283,2565042,Cerebellum,F8.7,76408
Cerebellum:F8.9:5930,46,3379,14026,20_13.msgp,2640475325,69617,Cerebellum,F8.9,5930
Cerebellum:F13.14:57107,203,91982,263072,20_13.msgp,2640544942,1387970,Cerebellum,F13.14,57107


In [20]:
psm_head = psm_head.set_index('File ID')

In [23]:
psm_index = pd.concat([psm_head,psm_index],axis=1,join='inner')

In [24]:
psm_index.to_csv('/home/z37mao/genova_filted_dataset_index.csv',index_label='Index')

In [32]:
np.any(psm_index['XCorr']==1)

False

In [33]:
np.any(psm_index['Percolator PEP']==0.05)

False

In [37]:
np.any(psm_index['Node Number']>=5e5)

False